In [1]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def parse_row_as_element(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    # return Element(name, prevElemCombos)
    prevElemCombos.sort()
    return {name : {'name': name, 'prevElemCombos': prevElemCombos, 'nextPossibleElements': []}}

def printx(listx):
    for x in listx:
        print(x)

def create_elements(csv_list):
    elements_arr = [parse_row_as_element(row) for row in csv_list]
    elements = elements_arr[0]
    for e_arr in elements_arr[1:]:
        elements.update(e_arr)
    return elements

In [2]:
filename = 'combinations_list_orig.csv'
csv_list = load_csv(filename)
elements = create_elements(csv_list)
# print(elements)

In [3]:
def create_tree(elements):
    elem_names = list(elements.keys())
    elements['BASE_ELEMENT'] = {'name': 'BASE_ELEMENTS', 'prevElemCombos': [], 'nextPossibleElements': []}
    for elem in elem_names:
        for prevElemCombo in elements[elem]['prevElemCombos']:
            for prevElem in prevElemCombo:
                try:
                    elements[prevElem]['nextPossibleElements'].append(elem)
                    elements[prevElem]['nextPossibleElements'].sort()
                except KeyError:
                    elements[prevElem] = {'name': prevElem, 'prevElemCombos': [], 'nextPossibleElements': [elem]}         

    return elements 

elements = create_tree(elements)

In [4]:
from functools import lru_cache
cache = lru_cache(maxsize=None)

# @cache

def get_element_score(element, elements, level=0):
    if level == 0:
        return len(elements[element]['nextPossibleElements'])
    else:
        return sum([get_element_score(elem, elements, level-1) for elem in elements[element]['nextPossibleElements']])

# @cache
def get_valid_combination_list(elements, strs=False):
    arr = []
    for elem in elements:
        for c, combo in enumerate(elements[elem]['prevElemCombos']):
            arr.append('+'.join(combo + [elem]))
    arr = list(set(arr))
    return arr if strs else [a.split('+') for a in arr] 
    
    

In [5]:
import numpy as np
def numMap_of_combos(elements):
    elems = list(elements.keys())
    valid_combo_arr = get_valid_combination_list(elements)
    a = []
    nMap = [[a for x in range(len(elems)+1)] for y in range(len(elems)+1)]
    for i, combo in enumerate(valid_combo_arr):
        nMap[elems.index(combo[0])][elems.index(combo[1])].append(elems.index(combo[2]))
    return nMap

numMap_of_combos(elements)

    

IndexError: list index out of range

In [ ]:
from joblib import Parallel, delayed

def get_combos(current_elements, elements):
    valid_combos_and_res = get_valid_combination_list(elements)
    valid_combos_set = set(['+'.join(c[:-1]) for c in valid_combos_and_res])
    

    test_combos = ['+'.join([a, b]) for a in current_elements for b in current_elements]
    test_combos = test_combos + ['+'.join([b, a]) for a in current_elements for b in current_elements]
    
    test_combos_set = set(test_combos)
    
    test_combos = list(valid_combos_set.intersection(test_combos_set))
    return test_combos


def get_next_best_element(current_elements, elements):
    valid_combos = get_combos(current_elements, elements)
    all_next_elements = [arr[-1] for arr in valid_combos]
    

    get_rankings = Parallel(n_jobs=-1)(delayed(get_element_score)(elem, elements) for elem in all_next_elements)

    all_next_elements = [[all_next_elements[i], get_rankings[i]] for i in range(len(all_next_elements))]

    all_next_elements = sorted(all_next_elements, key=lambda x: x[1], reverse=True)

    return all_next_elements

In [ ]:
base_elements = elements['BASE_ELEMENT']['nextPossibleElements']

current_elements = base_elements

get_next_best_element(current_elements, elements)

['air+air', 'air+earth', 'air+fire', 'air+water', 'earth+air', 'earth+earth', 'earth+fire', 'earth+water', 'fire+air', 'fire+earth', 'fire+fire', 'fire+water', 'water+air', 'water+earth', 'water+fire', 'water+water']


[]

In [ ]:
get_valid_combination_list(elements)

[['village', 'village', 'city'],
 ['meat', 'smoke', 'ham'],
 ['fish', 'star', 'starfish'],
 ['pumpkin', 'fire', "jack-o'-lantern"],
 ['wood', 'leather', 'drum'],
 ['leaf', 'fence', 'hedge'],
 ['sword', 'stone', 'excalibur'],
 ['air', 'wine', 'vinegar'],
 ['steam engine', 'metal', 'train'],
 ['wind', 'paper', 'kite'],
 ['bacteria', 'glasses', 'microscope'],
 ['oxygen', 'electricity', 'ozone'],
 ['egg', 'oil', 'mayonnaise'],
 ['fruit', 'time', 'alcohol'],
 ['sheep', 'tool', 'wool'],
 ['chocolate', 'caramel', 'caramelo'],
 ['flour', 'milk', 'batter'],
 ['horse', 'sea', 'seahorse'],
 ['chicken', 'wild animal', 'fox'],
 ['willy wonka', 'river', 'chocolate'],
 ['cold', 'yogurt', 'frozen yogurt'],
 ['human', 'firetruck', 'fireman'],
 ['sea', 'antarctica', 'iceberg'],
 ['wheel', 'wind', 'windmill'],
 ['coffin', 'earth', 'grave'],
 ['livestock', 'santa', 'reindeer'],
 ['human', 'bird', 'angel'],
 ['water', 'plant', 'algae'],
 ['livestock', 'house', 'barn'],
 ['glass', 'snow', 'snow globe'],
 ['

In [10]:
a = [[[] for i in range(3)] for j in range(3)]
a[2][1].append(1)
a

[[[], [], []], [[], [], []], [[], [1], []]]